In [27]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from s3 import S3
from depth_focus import apply_depth_based_gaussian
from cpbd.compute import *
from MMZ_paper import *
from scores import *
import pandas as pd

def normalize_float_image_for_saving(img, cmap='gray'):
    norm_img = (img * 255).astype(np.uint8)
    return norm_img

# Dataset tid2008 should be under the working directory

Load an RGBD image for depth blur (generating function is in depth_focus.py), an image for global blur, an image for motion blur.


In [18]:
# Load an RGBD image
rgb_image = cv.imread("rgb.jpg")
depth_image = cv.imread("depth.png", cv.IMREAD_GRAYSCALE)
depth_image = depth_image.astype(np.float32)
# Load anoter RGB image
image = cv.imread("tid2008/reference_images/I01.BMP")

Apply different blurring techniques

In [ ]:
# Gaussian blur images
blurred_gaussian_images = [image]
for sigma in [0.5, 1.0, 2.0]:
    blurred_image = cv.GaussianBlur(image, (0, 0), sigma)
    blurred_gaussian_images.append(blurred_image)

# Depth blur using the depth image
blurred_depth_images = [rgb_image]
for focused_depth_val in [50, 100, 200]:
    depth_blurred_image = apply_depth_based_gaussian(rgb_image, depth_image, focused_depth_val)
    blurred_depth_images.append(depth_blurred_image)

# Motion blur images
running_man_original = cv.imread("running.png", cv.IMREAD_GRAYSCALE)
running_man_blurred = cv.imread("running man blurred.jpg", cv.IMREAD_GRAYSCALE)
blurred_motion_images = [running_man_original, running_man_blurred]

# Using CPBD

In [ ]:
# Compute blur level estimations using CPBD
cpbd_results_blurred_gaussian_images = []
for img in blurred_gaussian_images:
    blurmap, score = compute_cpbd_blur_map_and_score(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    cpbd_results_blurred_gaussian_images.append(score)  # A single scalar value

cpbd_results_depth_images = []
for img in blurred_depth_images:
    blurmap, score = compute_cpbd_blur_map_and_score(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    cpbd_results_depth_images.append(score) # A single scalar value

cpbd_results_blurred_motion_images = []
for img in blurred_motion_images:
    blurmap, score = compute_cpbd_blur_map_and_score(img) # Assuming the image is already grayscale
    cpbd_results_blurred_motion_images.append(score) # A single scalar value

# Using MMZ

In [ ]:
cpbd_results_blurred_gaussian_images = []
for img in blurred_depth_images:
    _, padded_image = bd.check_image_size(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    sharpness_map = 1 - bd.get_blurness(padded_image)
    score = np.mean(sharpness_map)
   

for img in blurred_gaussian_images:
    _, padded_image = bd.check_image_size(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    sharpness_map = 1 - bd.get_blurness(padded_image)
    score = np.mean(sharpness_map)

for img in cpbd_results_blurred_motion_images:
    _, padded_image = bd.check_image_size(img)
    sharpness_map = 1 - bd.get_blurness(padded_image)
    score = np.mean(sharpness_map)

# Using S3

In [ ]:
# Compute blur level estimations using S3
s3 = S3()
s3_results_blurred_gaussian_images = []
for img in blurred_gaussian_images:
    result = s3.compute_s3(img) # s1, s2, s3 maps and the final score
    s3_results_blurred_gaussian_images.append(result)

s3_results_blurred_depth_images = []
for img in blurred_depth_images:
    result = s3.compute_s3(img) # s1, s2, s3 maps and the final score
    s3_results_blurred_depth_images.append(result)

s3_results_blurred_motion_images = []
for img in blurred_motion_images:
    result = s3.compute_s3(img) # s1, s2, s3 maps and the final score
    s3_results_blurred_motion_images.append(result)
    
results_folder = "results/s3_results/"
os.makedirs(results_folder, exist_ok=True)
for i, img in enumerate(blurred_gaussian_images):
    s3_score = s3_results_blurred_gaussian_images[i][3]  # Final score
    cv.imwrite(results_folder + f"blurred_gaussian_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_blurred_gaussian_{i}.png", normalize_float_image_for_saving(s3_results_blurred_gaussian_images[i][0]))
    cv.imwrite(results_folder + f"s2_blurred_gaussian_{i}.png", normalize_float_image_for_saving(s3_results_blurred_gaussian_images[i][1]))
    cv.imwrite(results_folder + f"s3_blurred_gaussian_{i}.png", normalize_float_image_for_saving(s3_results_blurred_gaussian_images[i][2]))

for i, img in enumerate(blurred_depth_images):
    s3_score = s3_results_blurred_depth_images[i][3]  # Final score
    cv.imwrite(results_folder + f"blurred_depth_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_blurred_depth_{i}.png", normalize_float_image_for_saving(s3_results_blurred_depth_images[i][0]))
    cv.imwrite(results_folder + f"s2_blurred_depth_{i}.png", normalize_float_image_for_saving(s3_results_blurred_depth_images[i][1]))
    cv.imwrite(results_folder + f"s3_blurred_depth_{i}.png", normalize_float_image_for_saving(s3_results_blurred_depth_images[i][2]))

for i, img in enumerate(blurred_motion_images):
    s3_score = s3_results_blurred_motion_images[i][3]  # Final score
    cv.imwrite(results_folder + f"blurred_motion_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_blurred_motion_{i}.png", normalize_float_image_for_saving(s3_results_blurred_motion_images[i][0]))
    cv.imwrite(results_folder + f"s2_blurred_motion_{i}.png", normalize_float_image_for_saving(s3_results_blurred_motion_images[i][1]))
    cv.imwrite(results_folder + f"s3_blurred_motion_{i}.png", normalize_float_image_for_saving(s3_results_blurred_motion_images[i][2]))

### Reproducing TID2008 global blur figures in the report

In [ ]:
# Read an image and blurred versions for TID2008 dataset
image_number = "I07"
original_image = cv.imread("tid2008/reference_images/{image_number}.BMP", cv.IMREAD_GRAYSCALE)
blurred_images = []
for i in range(1, 5):
    blurred_image = cv.imread(f"tid2008/distorted_images/{image_number}_08_{i}.bmp", cv.IMREAD_GRAYSCALE)
    blurred_images.append(blurred_image)

# Make a dataframe to store the results
df = pd.DataFrame(columns=["Blur Strength", "Score", "Algorithm"])

# Compute CPBD for the original and blurred images
cpbd_maps = []
for blur_level, img in enumerate([original_image] + blurred_images):
    map, score = compute_cpbd_blur_map_and_score(img)
    cpbd_maps.append(map)
    # Add score to the dataframe along with blur level
    df.loc[len(df)] = {"Blur Strength": blur_level, "Score": score, "Algorithm": "CPBD"}

# Save CPBD maps for the original and blurred images
cpbd_results_folder = "results/tid2008_cpbd_images/"
os.makedirs(cpbd_results_folder, exist_ok=True)
for i, img in enumerate([original_image] + blurred_images):
    cpbd_map = cpbd_maps[i]
    cv.imwrite(cpbd_results_folder + f"tid2008_image_{i}_cpbd.png", normalize_float_image_for_saving(cpbd_map))
    cv.imwrite(cpbd_results_folder + f"tid2008_image_{i}_cpbd_score_{df['Score'][i]:.2f}.png", img)

# Compute S3 maps for the original and blurred images
s3 = S3()
s3_results = []
for blur_level, img in enumerate([original_image] + blurred_images):
    result = s3.compute_s3(img)  # s1, s2, s3 maps and the final score
    s3_results.append(result)
    # Add score to the dataframe along with blur level
    df.loc[len(df)] = {"Blur Strength": blur_level, "Score": result[3], "Algorithm": "S3"}

# Save the results for S3 maps of TID2008 images
tid2008_results_folder = "results/tid2008_s3_images/"
os.makedirs(tid2008_results_folder, exist_ok=True)
for i, img in enumerate([original_image] + blurred_images):
    s3_score = s3_results[i][3]  # Final score
    cv.imwrite(tid2008_results_folder + f"tid2008_image_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(tid2008_results_folder + f"s1_tid2008_image_{i}.png", normalize_float_image_for_saving(s3_results[i][0]))
    cv.imwrite(tid2008_results_folder + f"s2_tid2008_image_{i}.png", normalize_float_image_for_saving(s3_results[i][1]))
    cv.imwrite(tid2008_results_folder + f"s3_tid2008_image_{i}.png", normalize_float_image_for_saving(s3_results[i][2]))

# Compute the MMZ score and maps
mmz_maps = []
bd = BlurDetector()
for blur_level, img in enumerate([original_image] + blurred_images):
    _, padded_image = bd.check_image_size(img)
    sharpness_map = 1 - bd.get_blurness(padded_image)
    score = np.mean(sharpness_map)
    mmz_maps.append(sharpness_map)
    # Add score to the dataframe along with blur level
    df.loc[len(df)] = {"Blur Strength": blur_level, "Score": score, "Algorithm": "MMZ"}

# Save MMZ maps for the original and blurred images
mmz_results_folder = "results/tid2008_mmz_images/"
os.makedirs(mmz_results_folder, exist_ok=True)
for i, img in enumerate([original_image] + blurred_images):
    mmz_map = mmz_maps[i]
    cv.imwrite(mmz_results_folder + f"tid2008_image_{i}_mmz.png", normalize_float_image_for_saving(mmz_map))
    cv.imwrite(mmz_results_folder + f"tid2008_image_{i}_mmz_score_{df['Score'][i]:.2f}.png", img)

# Save the dataframe to a CSV file
df.to_csv("results/tid2008_blur_scores.csv", index=False)
# Display the dataframe
print(df)


61
61
61
61
    Blur Strength     Score Algorithm
0               0  0.578534      CPBD
1               1  0.391636      CPBD
2               2  0.242802      CPBD
3               3  0.049812      CPBD
4               0  0.588528        S3
5               1  0.300445        S3
6               2  0.120586        S3
7               3  0.067587        S3
8               0  0.467142       MMZ
9               1  0.325200       MMZ
10              2  0.252351       MMZ
11              3  0.178368       MMZ


In [ ]:
# Computes spearman correlation scores for the algorithms
compute_correlation_scores()

61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61


### Reproducing motion blur images in the report

In [ ]:
# Compute blur level estimations using MMZ
bd = BlurDetector()
mmz_results_folder = "results/motion_blur_mmz_images/"
os.makedirs(mmz_results_folder, exist_ok=True)
for img in blurred_motion_images:
    _, padded_image = bd.check_image_size(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    sharpness_map = 1 - bd.get_blurness(padded_image)
    score = np.mean(sharpness_map)

# Save MMZ maps for the motion blurred images
for i, img in enumerate(blurred_motion_images):
    mmz_map = sharpness_map
    cv.imwrite(mmz_results_folder + f"motion_blurred_image_{i}_mmz.png", normalize_float_image_for_saving(mmz_map))
    cv.imwrite(mmz_results_folder + f"motion_blurred_image_{i}_mmz_score_{score:.2f}.png", img)

# Compute blur level estimation using S3 for motion blurred images
s3_results_motion_blurred_images = []
for img in blurred_motion_images:
    result = s3.compute_s3(img)  # s1, s2, s3 maps and the final score
    s3_results_motion_blurred_images.append(result)

# Save the results for S3 maps of motion blurred images
motion_blurred_results_folder = "results/motion_blurred_s3_images/"
os.makedirs(motion_blurred_results_folder, exist_ok=True)
for i, img in enumerate(blurred_motion_images):
    s3_score = s3_results_motion_blurred_images[i][3]  # Final score
    cv.imwrite(motion_blurred_results_folder + f"motion_blurred_image_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(motion_blurred_results_folder + f"s1_motion_blurred_image_{i}.png", normalize_float_image_for_saving(s3_results_motion_blurred_images[i][0]))
    cv.imwrite(motion_blurred_results_folder + f"s2_motion_blurred_image_{i}.png", normalize_float_image_for_saving(s3_results_motion_blurred_images[i][1]))
    cv.imwrite(motion_blurred_results_folder + f"s3_motion_blurred_image_{i}.png", normalize_float_image_for_saving(s3_results_motion_blurred_images[i][2]))

# Compute blur level estimation using CPBD for motion blurred images
cpbd_results_motion_blurred_images = []
for img in blurred_motion_images:
    blurmap, score = compute_cpbd_blur_map_and_score(img)  # Assuming the image is already grayscale
    cpbd_results_motion_blurred_images.append(score)  # A single scalar value

# Save CPBD maps for the motion blurred images
motion_blurred_cpbd_results_folder = "results/motion_blurred_cpbd_images/"
os.makedirs(motion_blurred_cpbd_results_folder, exist_ok=True)
for i, img in enumerate(blurred_motion_images):
    cv.imwrite(motion_blurred_cpbd_results_folder + f"motion_blurred_image_{i}_cpbd.png", normalize_float_image_for_saving(cpbd_results_motion_blurred_images[i]))
    cv.imwrite(motion_blurred_cpbd_results_folder + f"motion_blurred_image_{i}_cpbd_score_{cpbd_results_motion_blurred_images[i]:.2f}.png", img)

### Reproducing depth blur images in the report

In [ ]:
# Process depth of field blur image using all methods
blurred_depth_images = [rgb_image]
for focused_depth_val in [50, 100, 200]:
    depth_blurred_image = apply_depth_based_gaussian(rgb_image, depth_image, focused_depth_val)
    blurred_depth_images.append(depth_blurred_image)

# Compute blur level estimations using CPBD for depth of field images
cpbd_results_depth_images = []
for img in blurred_depth_images:
    result, score = compute_cpbd_blur_map_and_score(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    cpbd_results_depth_images.append(result)

# Save CPBD maps for depth of field images
cpbd_results_folder = "results/depth_of_field_cpbd_images/"
os.makedirs(cpbd_results_folder, exist_ok=True)
for i, img in enumerate(blurred_depth_images):
    cpbd_map = cpbd_results_depth_images[i]
    cv.imwrite(cpbd_results_folder + f"depth_of_field_image_{i}_cpbd.png", normalize_float_image_for_saving(cpbd_map))
    cv.imwrite(cpbd_results_folder + f"depth_of_field_image_{i}_cpbd_score_{score:.2f}.png", img)

# Compute S3 maps for depth of field images
s3_results_depth_images = []
for img in blurred_depth_images:
    result = s3.compute_s3(img)  # s1, s2, s3 maps and the final score
    s3_results_depth_images.append(result)

# Save the results for S3 maps of depth of field images
tid2008_results_folder = "results/depth_of_field_s3_images/"
os.makedirs(tid2008_results_folder, exist_ok=True)
for i, img in enumerate(blurred_depth_images):
    s3_score = s3_results_depth_images[i][3]  # Final score
    cv.imwrite(tid2008_results_folder + f"depth_of_field_image_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(tid2008_results_folder + f"s1_depth_of_field_image_{i}.png", normalize_float_image_for_saving(s3_results_depth_images[i][0]))
    cv.imwrite(tid2008_results_folder + f"s2_depth_of_field_image_{i}.png", normalize_float_image_for_saving(s3_results_depth_images[i][1]))
    cv.imwrite(tid2008_results_folder + f"s3_depth_of_field_image_{i}.png", normalize_float_image_for_saving(s3_results_depth_images[i][2]))

# Compute the MMZ score and maps for depth of field images
mmz_maps_depth = []
bd = BlurDetector()
for blur_level, img in enumerate(blurred_depth_images):
    _, padded_image = bd.check_image_size(img)
    sharpness_map = 1 - bd.get_blurness(padded_image)
    score = np.mean(sharpness_map)
    mmz_maps_depth.append(sharpness_map)

# Save MMZ maps for depth of field images
mmz_results_folder_depth = "results/depth_of_field_mmz_images/"
os.makedirs(mmz_results_folder_depth, exist_ok=True)
for i, img in enumerate(blurred_depth_images):
    mmz_map = mmz_maps_depth[i]
    cv.imwrite(mmz_results_folder_depth + f"depth_of_field_image_{i}_mmz.png", normalize_float_image_for_saving(mmz_map))
    cv.imwrite(mmz_results_folder_depth + f"depth_of_field_image_{i}_mmz_score_{score:.2f}.png", img)